In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import scipy.spatial.distance

In [3]:
import alphatsp
import alphatsp.tsp

In [4]:
def nearest_insertion(tsp):
    
    distances = scipy.spatial.distance.pdist(tsp.points, metric="euclidean")
    distances = scipy.spatial.distance.squareform(distances)
    np.fill_diagonal(distances, np.inf)

    adj = np.zeros((tsp.n, tsp.n), dtype=np.bool)
    start_node = np.argmax(distances[0, 1:]) + 1
    adj[0, start_node] = 1
    adj[start_node, 0] = 1

    nontour_nodes = set(range(tsp.n))
    tour_nodes = list()

    nontour_nodes.remove(0)
    nontour_nodes.remove(start_node)
    tour_nodes.append(0)
    tour_nodes.append(start_node)

    while nontour_nodes:
        
        best_dist = np.inf
        best_edge = (-1, -1)
        
        for i in nontour_nodes:
            
            dists = distances[i, tour_nodes]
            opt_ind = np.argmin(dists)
            opt_dist = dists[opt_ind]
            opt_node = tour_nodes[opt_ind]
            
            if opt_dist < best_dist:
                best_dist = opt_dist
                best_edge = (i, opt_node)
                
        i, j = best_edge
                
        tour_nodes.append(i)
        nontour_nodes.remove(i)
        
        egde_in = np.argmax(adj[:,j])
        edge_out = np.argmax(adj[j,:])
        
        inc1 = distances[egde_in, i] + distances[i, j] - distances[egde_in, j]
        inc2 = distances[j, i] + distances[i, edge_out] - distances[j, edge_out]
        
        if inc1 <= inc2:
            adj[egde_in, j] = 0
            adj[egde_in, i] = 1
            adj[i, j] = 1
        else:
            adj[j, edge_out] = 0
            adj[i, edge_out] = 1
            adj[j, i] = 1
            
    tour = [0]
    while len(tour) <= tsp.n:
        tour.append(np.argmax(adj[tour[-1], :]))

    return tour

In [5]:
tsp = alphatsp.tsp.TSP(100, 2)

In [6]:
optimal_tour = nearest_insertion(tsp)
optimal_distance = tsp.tour_length(optimal_tour)

In [7]:
print(optimal_tour)
print(optimal_distance)

[0, 92, 29, 43, 4, 3, 33, 97, 41, 84, 88, 30, 10, 71, 56, 26, 45, 83, 90, 38, 67, 87, 22, 40, 7, 32, 62, 99, 18, 36, 63, 70, 89, 13, 42, 49, 55, 54, 39, 16, 23, 53, 47, 60, 52, 25, 68, 78, 86, 11, 61, 80, 69, 91, 94, 15, 14, 58, 96, 48, 6, 64, 35, 66, 31, 59, 74, 19, 93, 12, 24, 51, 9, 17, 27, 81, 1, 8, 2, 95, 46, 50, 28, 21, 65, 75, 57, 76, 98, 37, 44, 73, 85, 5, 20, 77, 72, 82, 34, 79, 0]
10.537137610992794
